In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Head_and_Neck_Cancer"
cohort = "GSE104006"

# Input paths
in_trait_dir = "../../input/GEO/Head_and_Neck_Cancer"
in_cohort_dir = "../../input/GEO/Head_and_Neck_Cancer/GSE104006"

# Output paths
out_data_file = "../../output/preprocess/Head_and_Neck_Cancer/GSE104006.csv"
out_gene_data_file = "../../output/preprocess/Head_and_Neck_Cancer/gene_data/GSE104006.csv"
out_clinical_data_file = "../../output/preprocess/Head_and_Neck_Cancer/clinical_data/GSE104006.csv"
json_path = "../../output/preprocess/Head_and_Neck_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
# Based on the series title which mentions "miRNA and gene expression profiling", 
# we can infer that gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait: Looking at key 0, we see disease status which matches our Head_and_Neck_Cancer trait
trait_row = 0

# For age: The age information is available in key 2
age_row = 2

# For gender: Gender information is available in key 3 as "Sex"
gender_row = 3

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert trait value to binary format (0 for control, 1 for case)"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon and strip spaces
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # For Head and Neck Cancer, map thyroid carcinoma to case (1) and non-neoplastic to control (0)
    if "Thyroid_carcinoma" in value or "carcinoma" in value.lower():
        return 1
    elif "Non-neoplastic" in value:
        return 0
    else:
        return None

def convert_age(value):
    """Convert age value to continuous format"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon and strip spaces
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender value to binary format (0 for female, 1 for male)"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon and strip spaces
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value.upper() == 'F':
        return 0
    elif value.upper() == 'M':
        return 1
    else:
        return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features using the library function
    clinical_features_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(clinical_features_df)
    print("Preview of clinical features dataframe:")
    print(preview)
    
    # Save the clinical data
    clinical_features_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 4: Gene Identifier Review

In [ ]:
# Analyzing the gene identifiers in the dataset

# Looking at the first 20 identifiers, they include:
# - 'Blank' (likely control spots)
# - 'dmr_X' prefixed identifiers (likely custom probes)
# - 'hsa-let-X' prefixed identifiers (these are human microRNA identifiers)

# These identifiers appear to be microRNA (miRNA) names/IDs rather than standard human gene symbols
# The 'hsa-' prefix indicates human (Homo sapiens) miRNAs
# This is a miRNA profiling dataset, not a standard gene expression dataset

# Since these are already standard miRNA identifiers, no mapping to gene symbols is needed
# We would use these miRNA identifiers directly in analysis

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [ ]:
# 1. Save the gene data (miRNA data in this case) since we determined no mapping is needed
print(f"Gene data shape: {gene_data.shape}")

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")

# 2. Use the clinical data we extracted in step 2
if 'clinical_features_df' not in locals():
    # Get clinical data from matrix file again if needed
    background_info, clinical_data = get_background_and_clinical_data(matrix_file)
    
    # Define proper conversion functions for the trait
    def convert_trait(value):
        """Convert histology information to binary trait values (0: control, 1: cancer)"""
        if not isinstance(value, str):
            return None
        value = value.lower().split(': ')[-1]  # Extract value after colon
        
        # Map to binary: 1 for any tumor tissue (PDTC/PTC variants), 0 for non-neoplastic
        if 'non-neoplastic_thyroid' in value:
            return 0  # Control
        elif 'ptc' in value or 'pdtc' in value:
            return 1  # Cancer
        else:
            return None  # Unclear
    
    # Use the trait_row, age_row, and gender_row from step 2
    clinical_features_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
# Print clinical data shape and preview
print("Clinical data shape:", clinical_features_df.shape)
print("Clinical data preview:", preview_df(clinical_features_df))

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features_df, gene_data)
print(f"Linked data shape before handling missing values: {linked_data.shape}")

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 5. Evaluate bias in trait and demographic features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Conduct final quality validation
note = "Dataset contains thyroid carcinoma expression data (PDTC/PTC), which is relevant for head and neck cancer. This is a miRNA dataset."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 7. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable due to quality issues - linked data not saved")